In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()
time_now_formatted = time_now.strftime("%Y-%m-%d")

In [4]:
data=pd.read_sql_query(
    f"Select DISTINCT Winner,Dog,Dog_Record,Winner_Odds,Fav_Record,percent,percent2,Fav_Games,Dog_Games,Fav_Odds,Fav,sex From results_hard \
    where sex like 'Mens' and Date > '2023-01-01'",
    con=devengine,)
data['Fav_Odds']=data['Fav_Odds'].astype(float)
data['Winx']=data.apply(lambda x: 'Dog' if x['Winner']==x['Dog'] else 'Fav', axis=1)
#print(len(data[(data['Winner']==data['Fav'])&(data['Fav_Odds']>1.9)])/len(data[data['Fav_Odds']>1.9]))
#data[data['Fav_Odds']>1.8].to_csv('test.csv')
#data=data[data['Fav_Odds']>1.19]
#data=data[data['percent']>0.3]
#data=data[data['percent2']>0.1]

In [3]:
mensserving=pd.read_csv('mensserving.csv')
combine = pd.merge(data, mensserving, how="left", left_on="Fav", right_on="Name")
combine2 = pd.merge(combine, mensserving, how="left", left_on="Dog", right_on="Name")
combine2[["Service Games Won_x", "Service Games Won_y"]] = combine2[
        ["Service Games Won_x", "Service Games Won_y"]
    ].astype(float)
combine2.rename(
        columns={
            "Service Games Won_x": "Fav_Serve%",
            "Service Games Won_y": "Dog_Serve%",
            "Return Games Won_x": "Fav_Return%",
            "Return Games Won_y": "Dog_Return%",
        },
        inplace=True,
    )
mens = combine2[
    [   "Winner","Dog","Winner_Odds","percent","percent2","Fav_Games","Dog_Games","Fav_Odds",
        "Fav",
        # "Player_1_Odds",
        "Fav_Serve%",
        "Dog_Return%",
       
        # "Player_2_Odds",
        "Dog_Serve%",
        "Fav_Return%",
    ]]

In [4]:
mensfilter=mens[(mens['percent2']>0)&(mens['percent']>0.5)&(mens['Fav_Serve%']>70)&(mens['Dog_Serve%']>0.0)&(mens['Dog_Serve%']<70.0)&(mens['Dog_Return%']>0)]#&(combine3['Fav_Odds']>1.8)]
print(len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter))
print(len(mensfilter))

0.7857142857142857
28


In [5]:
import numpy as np

print("><")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
   
        mensfilter=mens[(mens['percent2']>x)&(mens['percent']<y)]
        
        if len(mensfilter)>5 and len(mensfilter[mensfilter['Winner']==mensfilter['Dog']])/len(mensfilter) > 0.5:

            print(x,y)
            print(len(mensfilter[mensfilter['Winner']==mensfilter['Dog']])/len(mensfilter))
            print(len(mensfilter))
print("<<")            
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
   
        mensfilter=mens[(mens['percent2']<x)&(mens['percent']<y)]
        
        if len(mensfilter)>10 and len(mensfilter[mensfilter['Winner']==mensfilter['Dog']])/len(mensfilter) > 0.5:
            
            print(x,y)
            print(len(mensfilter[mensfilter['Winner']==mensfilter['Dog']])/len(mensfilter))
            print(len(mensfilter))      
print(">>")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        
   
        mensfilter=mens[(mens['percent2']>x)&(mens['percent']>y)]
        if len(mensfilter)>10 and len(mensfilter[mensfilter['Winner']==mensfilter['Dog']])/len(mensfilter) > 0.5:
            print(x,y)
            print(len(mensfilter[mensfilter['Winner']==mensfilter['Dog']])/len(mensfilter))
            print(len(mensfilter))       
print("<>")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        
   
        mensfilter=mens[(mens['percent2']<x)&(mens['percent']>y)]
        if len(mensfilter)>10 and len(mensfilter[mensfilter['Winner']==mensfilter['Dog']])/len(mensfilter) > 0.5:
            print(x,y)
            print(len(mensfilter[mensfilter['Winner']==mensfilter['Dog']])/len(mensfilter))
            print(len(mensfilter))    

><
0.0 0.30000000000000004
0.5161290322580645
31
0.1 0.30000000000000004
0.5161290322580645
31
0.2 0.30000000000000004
0.5769230769230769
26
0.30000000000000004 0.30000000000000004
0.5416666666666666
24
0.4 0.30000000000000004
0.5384615384615384
13
<<
>>
<>


In [6]:
import numpy as np

print("><")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
   
        mensfilter=mens[(mens['percent2']>x)&(mens['percent']<y)&(mens['Fav_Odds']>1.19)&(mens['Fav_Odds']<1.3)]
        
        if len(mensfilter)>10 and len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter) > 0.9:

            print(x,y)
            print(len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter))
            print(len(mensfilter))
print("<<")            
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
   
        mensfilter=mens[(mens['percent2']<x)&(mens['percent']<y)&(mens['Fav_Odds']>1.19)&(mens['Fav_Odds']<1.3)]
        
        if len(mensfilter)>10 and len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter) > 0.9:
            
            print(x,y)
            print(len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter))
            print(len(mensfilter))      
print(">>")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        
   
        mensfilter=mens[(mens['percent2']>x)&(mens['percent']>y)&(mens['Fav_Odds']>1.19)&(mens['Fav_Odds']<1.3)]
        if len(mensfilter)>10 and len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter) > 0.9:
            print(x,y)
            print(len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter))
            print(len(mensfilter))       
print("<>")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        
   
        mensfilter=mens[(mens['percent2']<x)&(mens['percent']>y)&(mens['Fav_Odds']>1.19)&(mens['Fav_Odds']<1.3)]
        if len(mensfilter)>10 and len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter) > 0.9:
            print(x,y)
            print(len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter))
            print(len(mensfilter))    

><
<<
>>
0.0 0.8
0.9375
16
0.1 0.8
0.9375
16
0.2 0.8
1.0
13
0.30000000000000004 0.8
1.0
13
0.4 0.8
1.0
11
<>
0.6000000000000001 0.8
0.9090909090909091
11
0.7000000000000001 0.8
0.9230769230769231
13
0.8 0.8
0.9230769230769231
13
0.9 0.8
0.9333333333333333
15


In [7]:
mensfilter=mens[(mens['Fav_Odds']>1.8)]
print(len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter))
print(len(mensfilter))

0.6274509803921569
51


In [8]:
data=pd.read_sql_query(
    f"Select DISTINCT Winner,Dog,Dog_Record,Winner_Odds,Fav_Record,percent,percent2,Fav_Games,Dog_Games,Fav_Odds,Fav,sex From results_hard where sex like 'Womens' --where Date > '2023-01-01' and Date not like '{time_now_formatted}'",
    con=devengine,)
data['Fav_Odds']=data['Fav_Odds'].astype(float)
data['Winx']=data.apply(lambda x: 'Dog' if x['Winner']==x['Dog'] else 'Fav', axis=1)

In [9]:
womensserving=pd.read_csv('womensserving.csv')
combine = pd.merge(data, womensserving, how="left", left_on="Fav", right_on="Name")
combine2 = pd.merge(combine, womensserving, how="left", left_on="Dog", right_on="Name")
combine2[["Service Games Won_x", "Service Games Won_y"]] = combine2[
        ["Service Games Won_x", "Service Games Won_y"]
    ].astype(float)
combine2.rename(
        columns={
            "Service Games Won_x": "Fav_Serve%",
            "Service Games Won_y": "Dog_Serve%",
            "Return Games Won_x": "Fav_Return%",
            "Return Games Won_y": "Dog_Return%",
        },
        inplace=True,
    )
womens = combine2[
    [   "Winner","Dog","Winner_Odds","percent","percent2","Fav_Games","Dog_Games","Fav_Odds",
        "Fav",
        # "Player_1_Odds",
        "Fav_Serve%",
        "Dog_Return%",
       
        # "Player_2_Odds",
        "Dog_Serve%",
        "Fav_Return%",
    ]]

In [10]:
womensfilter=womens[(womens['percent2']>0)&(womens['percent']>0.5)&(womens['Fav_Serve%']>70)&(womens['Dog_Serve%']>0.0)&(womens['Dog_Serve%']<70.0)&(womens['Dog_Return%']>0)&(womens['Fav_Odds']>1.8)]
len(womensfilter[womensfilter['Winner']!=womensfilter['Dog']])/len(womensfilter)

0.0

In [11]:
len(womensfilter)

1

In [12]:
data3=womens[(womens['Fav_Odds']>1.19)&(womens['Fav_Odds']<1.3)&(womens['percent2']<0.3)&(womens['percent']>0.6)&(womens['Dog_Games']>8)]
len(data3[data3['Winner']==data3['Fav']])/len(data3)

1.0

In [13]:
len(data3)

4

In [14]:
data3

,Winner,Dog,Winner_Odds,percent,percent2,Fav_Games,Dog_Games,Fav_Odds,Fav,Fav_Serve%,Dog_Return%,Dog_Serve%,Fav_Return%
91,Cori Gauff,Emma Raducanu,1.29,0.827586,0.100000,29,10,1.29,Cori Gauff,NaN,38.9,68.5,NaN
127,Petra Kvitova,Alison Van Uytvanck,1.24,0.647059,0.111111,34,9,1.24,Petra Kvitova,74.3,32.2,71.5,29.9
287,Kaia Kanepi,Anna Bondar,1.28,0.650000,0.214286,20,14,1.28,Kaia Kanepi,71.7,30.3,70.3,31.8
324,Cori Gauff,Sofia Kenin,1.21,0.818182,0.272727,11,11,1.21,Cori Gauff,NaN,23.3,63.5,NaN


In [15]:
import numpy as np

print("><")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
   
        womensfilter=womens[(womens['percent2']>x)&(womens['percent']<y)&(womens['Fav_Odds']>1.19)&(womens['Fav_Odds']<1.3)]
        
        if len(womensfilter)>10 and len(womensfilter[womensfilter['Winner']!=womensfilter['Dog']])/len(womensfilter) > 0.9:

            print(x,y)
            print(len(womensfilter[womensfilter['Winner']!=womensfilter['Dog']])/len(womensfilter))
            print(len(womensfilter))
print("<<")            
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
   
        womensfilter=womens[(womens['percent2']<x)&(womens['percent']<y)&(womens['Fav_Odds']>1.19)&(womens['Fav_Odds']<1.3)]
        
        if len(womensfilter)>10 and len(womensfilter[womensfilter['Winner']!=womensfilter['Dog']])/len(womensfilter) > 0.9:
            
            print(x,y)
            print(len(womensfilter[womensfilter['Winner']!=womensfilter['Dog']])/len(womensfilter))
            print(len(womensfilter))      
print(">>")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        
   
        womensfilter=womens[(womens['percent2']>x)&(womens['percent']>y)&(womens['Fav_Odds']>1.19)&(womens['Fav_Odds']<1.3)]
        if len(womensfilter)>10 and len(womensfilter[womensfilter['Winner']!=womensfilter['Dog']])/len(womensfilter) > 0.9:
            print(x,y)
            print(len(womensfilter[womensfilter['Winner']!=womensfilter['Dog']])/len(womensfilter))
            print(len(womensfilter))       
print("<>")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        
   
        womensfilter=womens[(womens['percent2']<x)&(womens['percent']>y)&(womens['Fav_Odds']>1.19)&(womens['Fav_Odds']<1.3)]
        if len(womensfilter)>10 and len(womensfilter[womensfilter['Winner']!=womensfilter['Dog']])/len(womensfilter) > 0.9:
            print(x,y)
            print(len(womensfilter[womensfilter['Winner']!=womensfilter['Dog']])/len(womensfilter))
            print(len(womensfilter))    

><
0.2 0.7000000000000001
0.9130434782608695
23
0.30000000000000004 0.7000000000000001
0.9047619047619048
21
<<
>>
<>
0.4 0.6000000000000001
0.9230769230769231
13


In [16]:
import numpy as np

print("><")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
   
        womensfilter=womens[(womens['percent2']>x)&(womens['percent']<y)]
        
        if len(womensfilter)>5 and len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter) > 0.5:

            print(x,y)
            print(len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter))
            print(len(womensfilter))
print("<<")            
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
   
        womensfilter=womens[(womens['percent2']<x)&(womens['percent']<y)]
        
        if len(womensfilter)>10 and len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter) > 0.5:
            
            print(x,y)
            print(len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter))
            print(len(womensfilter))      
print(">>")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        
   
        womensfilter=womens[(womens['percent2']>x)&(womens['percent']>y)]
        if len(womensfilter)>10 and len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter) > 0.5:
            print(x,y)
            print(len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter))
            print(len(womensfilter))       
print("<>")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        
   
        womensfilter=womens[(womens['percent2']<x)&(womens['percent']>y)]
        if len(womensfilter)>10 and len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter) > 0.5:
            print(x,y)
            print(len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter))
            print(len(womensfilter))    

><
<<
>>
0.5 0.7000000000000001
0.5333333333333333
15
0.6000000000000001 0.6000000000000001
0.5294117647058824
17
<>


In [17]:
womensfilter=womens[(womens['percent2']>0.2)&(womens['percent']<0.7)&(womens['Fav_Odds']>1.19)&(womens['Fav_Odds']<1.3)]
len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter)


0.10526315789473684

In [19]:
womens[(womens['percent2']>0.6)&(womens['percent']<0.4)&(womens['Fav_Odds']<1.8)]

,Winner,Dog,Winner_Odds,percent,percent2,Fav_Games,Dog_Games,Fav_Odds,Fav,Fav_Serve%,Dog_Return%,Dog_Serve%,Fav_Return%
5,Lesia Tsurenko,Ysaline Bonaventure,1.59,0.388889,0.615385,18,13,1.59,Lesia Tsurenko,60.7,29.1,72.2,43.6
148,Elisabetta Cocciaretto,Elisabetta Cocciaretto,2.49,0.363636,1.000000,22,5,1.54,Sofia Kenin,63.5,37.7,65.8,23.3
159,Lucrezia Stefanini,Lucrezia Stefanini,1.99,0.200000,0.625000,10,8,1.77,Sachia Vickery,33.3,28.3,57.4,22.2
199,Sofia Kenin,Maryna Zanevska,1.65,0.294118,0.857143,17,7,1.65,Sofia Kenin,63.5,36.3,63.2,23.3
